<a href="https://colab.research.google.com/github/Luca-Skyline/check4mate/blob/main/Chess_Piece_CNN_Model_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

10/29/2023 Luca DalCanto

# Chess Piece Convolutional Neural Network (CNN)
The code contained in this jupyter notebook was used to create the Deep Learning image classification model used in check4mate, an app in development by Luca DalCanto.

This code uses YOLOv5, a leading Neural Network for object detection and segmentation, but uses it instead for image classification. A dataset with around 25000 images is loaded from Roboflow, with each image labeled as one of 13 classes (6 pieces of two colors, plus the "empty square" class). The "medium" size of YOLOv5 model is fit with this data over 40 epochs, and the .pt file is then exported for use in my app.

### Data

You can find the dataset I used on Roboflow: https://universe.roboflow.com/luca-dalcanto-lrlwg/chess-piece-detector-sv3nm/dataset/4. This is my Roboflow project, but the images themselves are attributed to someone else (see below).

This data has been preprocessed and augmented with the following properties:

*   Preprocessing: Grayscale
*   Preprocessing: Resize to 224x224 pixels (this should not do anything with my particular dataset but it's a good safety)
*   Augmentation: 90°, 180°, 270° Rotation
*   Augmentation: Brightness 20%
*   Augmentation: Exposre 4%
*   Augmentation: Blur 0.3px

The preprocessing homogenizes the images, and the augmentation allows to have more images by duplicating some of the images with changes such as rotation and shear (the CNN needs to be able to identify a chess piece from any overhead angle).

Data attributed to Daylen Yang under the Open Data Commons Attribution License: http://opendatacommons.org/licenses/by/1.0/.
Thank you so much Daylen Yang!

This data has been adapted for this specific dataset. By the nature of the Open Data Commons Attribution License, this "new" dataset found here is also protected by the same license. This means you can use this data, but you need to attribute and any modified form of the data must also be released under the same license.

For more info on the raw data, please see Daylen Yang's github repo: https://github.com/daylen/chess-id

Thanks! <br>
Luca DalCanto, <br>
Skyline High School, SLC, UT

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/78.2 GB disk)


In [2]:
# Ensure we're in the right directory to download our custom dataset
import os
os.makedirs("../datasets/", exist_ok=True)
%cd ../datasets/

/content/datasets


In [3]:
## For safety reason, as this is a public colab, the API Key has been removed. To proceed, copy and paste the download code from Roboflow into this cell
# https://universe.roboflow.com/luca-dalcanto-lrlwg/chess-piece-detector-sv3nm/dataset/4

# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="INSERT KEY HERE")
# project = rf.workspace("luca-dalcanto-lrlwg").project("chess-piece-detector-sv3nm")
# version = project.version(9)
# dataset = version.download("folder")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.1

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Chess-Piece-Detector-9 in folder:: 100%|██████████| 4382/4382 [00:00<00:00, 9568.92it/s]


In [4]:
#Save the dataset name to the environment so we can use it in a system call later
dataset_name = dataset.location.split(os.sep)[-1]
os.environ["Chess-Piece-Detector-9"] = dataset_name

In [5]:
from genericpath import isfile
import os



for my_dir in os.listdir('/content/datasets/Chess-Piece-Detector-9/train'):
  num_files = 0
  for filename in os.listdir('/content/datasets/Chess-Piece-Detector-9/train/' + my_dir):
    num_files += 1
  print(my_dir,': ', num_files)


empty :  259
bk :  207
bn :  310
bq :  276
br :  197
bp :  288
wr :  288
wq :  311
wb :  315
wp :  268
wn :  300
wk :  208
bb :  323


***NOTE: Rename the "valid" folder to "val" at this time!***

In [8]:
%cd ../yolov5
!python classify/train.py --model yolov5m-cls.pt --data Chess-Piece-Detector-9 --epochs 200 --img 224 --pretrained weights/yolov5m-cls.pt

/content/yolov5
2024-03-17 14:34:36.055375: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 14:34:36.055430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 14:34:36.057110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
classify/train: model=yolov5m-cls.pt, data=Chess-Piece-Detector-9, epochs=200, batch_size=64, imgsz=224, nosave=False, cache=None, device=, workers=8, project=runs/train-cls, name=exp, exist_ok=False, pretrained=weights/yolov5m-cls.pt, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, lo

In [ ]:
!python classify/predict.py --weights /content/yolov5/runs/train-cls/exp2/weights/best.pt --source /content/somePiece.png

classify/predict: weights=['/content/yolov5/runs/train-cls/exp2/weights/best.pt'], source=/content/somePiece.png, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-283-g875d9278 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Traceback (most recent call last):
  File "/content/yolov5/classify/predict.py", line 238, in <module>
    main(opt)
  File "/content/yolov5/classify/predict.py", line 233, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/content/yolov5/classify/predict.py", line 102, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
  File "/content/yolov5/models/common.py", line 370, in __init__
    model

In [ ]:
!python export.py --weights yolov5s-cls.pt resnet50.pt efficientnet_b0.pt --include onnx engine --img 224